In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# --- Load Data ---
record_id = '418'
X = np.load(f'processed_data/{record_id}_segments.npy')
y = np.load(f'processed_data/{record_id}_labels.npy', allow_pickle=True)

# --- Encode Labels ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Optional: show class mappings
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label encoding:", label_map)

# --- Train/Test Split ---
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# --- Build CNN Model ---
input_shape = (2000, 2)

model = models.Sequential([
    layers.Conv1D(16, 7, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(2),
    
    layers.Conv1D(32, 5, activation='relu'),
    layers.MaxPooling1D(2),
    
    layers.Conv1D(64, 3, activation='relu'),
    layers.GlobalAveragePooling1D(),
    
    layers.Dense(32, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # multi-class output
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Label encoding: {'N': np.int64(0), 'VFL': np.int64(1)}


/opt/anaconda3/envs/ecg2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1994, 16)       │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 997, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 993, 32)        │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 496, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 494, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,186 (43.70 KB)

 Trainable params: 11,186 (43.70 KB)

 Non-trainable params: 0 (0.00 B)

: 

In [ ]:

# --- Train Model ---
history = model.fit(X_train, y_train, epochs=15, batch_size=32,
                    validation_data=(X_val, y_val))


Epoch 1/15


In [ ]:

# --- Save the Model and Label Map ---
model.save(f'models/ecg_cnn_{record_id}.h5')
np.save(f'models/{record_id}_label_classes.npy', label_encoder.classes_)
